In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import os
from ccfj import GetStationPairs
from geopy.distance import great_circle
#import shapefile
import geopandas as gp
import yaml
import math

from scipy.spatial import ConvexHull
from scipy.interpolate import griddata
from shapely.geometry import Point, Polygon

from pykrige.ok import OrdinaryKriging


In [ ]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

In [ ]:
flag_project = 1 # 0--regular; 1--repartrition

In [ ]:
if flag_project == 0:
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader) 
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    #name_project = 'project_repartrition/repartrition_01-03/' 
name_project = proj['name']              # Harmon server

In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
flag_fund = 1

In [ ]:
if flag_fund == 0:
    dir_model = dir_project + info_basic['dir_model_fund']
    dir_image = dir_project + info_basic['dir_image']+'Vs_fund/'
else:
    dir_model = dir_project + info_basic['dir_model']
    dir_image = dir_project + info_basic['dir_image']+'Vs_over/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)
dir_inv_dispernet = dir_project + info_basic['dir_inv_dispernet']
dir_partition = dir_project + info_basic['dir_partition']
key_subworks = info_basic['key_subworks']
M = len(key_subworks)
key_subworks

In [ ]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 

In [ ]:
stations_partition = {}
lat_stations_partition = {}
lon_stations_partition = {}
lat_centroid_partition = []
lon_centroid_partition = []
num_stations = []
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_partition[key] = stations_this
    lat_stations_partition[key] = lat_stations_this.astype(float)
    lon_stations_partition[key] = lon_stations_this.astype(float)
    num_stations.append(len(stations_this))
    lat_centroid_partition.append(np.mean(lat_stations_this.astype(float)))
    lon_centroid_partition.append(np.mean(lon_stations_this.astype(float)))

In [ ]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

### Interpolate 3D structure

In [ ]:
struc = {}
flag = 0
for key_subwork in key_subworks:
    file_model = dir_model + 'model_'+key_subwork+'.txt'
    model = np.loadtxt(file_model)
    struc[key_subwork] = {}
    struc[key_subwork]['layer'] = model[:, 0]
    struc[key_subwork]['z'] = model[:, 1]
    struc[key_subwork]['rho'] = model[:, 2]
    struc[key_subwork]['vs'] = model[:, 3]
    struc[key_subwork]['vp'] = model[:, 4]
    struc[key_subwork]['std'] = model[:, 5]
    flag += 1
    print('Read structure model: %s (%d/%d)' % (key_subwork, flag, M))

vs_horizon = {}
dz = 5
N = 40
for i in range(N):
    vs_horizon[i] = []
    for key_subwork in key_subworks:
        vs_horizon[i].append(struc[key_subwork]['vs'][i])

In [ ]:
num_lons = 100
num_lats = 100
dz = 5
N = 40
type_interp = 'linear'

v_min = 0.3
v_max = 0.65

In [ ]:
# generate grid
xx = np.linspace(np.min(lon_centroid_partition), np.max(lon_centroid_partition), num_lons)
yy = np.linspace(np.min(lat_centroid_partition), np.max(lat_centroid_partition), num_lats)
A,B = np.meshgrid(xx, yy)
X_star = np.hstack((A.flatten()[:,None], B.flatten()[:,None]))
lon_grid = X_star[:,0]
lat_grid = X_star[:,1]
# cut
points = np.column_stack((lon_centroid_partition, lat_centroid_partition))
hull = ConvexHull(points)
polygon = Polygon(points[hull.vertices])
index = []
lon_inter_in = []
lat_inter_in = []
for i in range(len(lon_grid)):
    if polygon.contains(Point(lon_grid[i], lat_grid[i])):
        index.append(i)
        lon_inter_in.append(lon_grid[i])
        lat_inter_in.append(lat_grid[i])

In [ ]:
vs_inter_horizon_all = {}
vs_inter_horizon_in = {}
for i in range(N):
    print('Interpolating horizon %d' % i)
    OK = OrdinaryKriging(lon_centroid_partition, lat_centroid_partition, vs_horizon[i], variogram_model=type_interp,nlags=3)
    zz,ss = OK.execute('grid', xx, yy)
    vs_inter_horizon_all[i] = zz.reshape(len(xx)*len(yy))
    vs_inter_horizon_in[i] = zz.reshape(len(xx)*len(yy))[index]

In [ ]:
for i in range(N):
    fig = plt.figure(figsize=(20, 7))
    ax = fig.add_subplot(121)
    ax.scatter(lon_stations_all, lat_stations_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Fundamental')
    flag_add = 0.0005
    ax.set_xlim([min(lon_stations_all)-flag_add, max(lon_stations_all)+flag_add])
    ax.set_ylim([min(lat_stations_all)-flag_add, max(lat_stations_all)+flag_add])
    for j in range(len(faults)):
        ax.plot(faults['clark'+str(j+1)]['lon'], faults['clark'+str(j+1)]['lat'], 'k')
    im = ax.scatter(lon_centroid_partition,lat_centroid_partition,c=vs_horizon[i],cmap='gist_rainbow',s=100)
    plt.colorbar(im)
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值

    ax = fig.add_subplot(122)
    ax.scatter(lon_stations_all, lat_stations_all,marker='.',color='k')
    for j in range(len(faults)):
        ax.plot(faults['clark'+str(j+1)]['lon'], faults['clark'+str(j+1)]['lat'], 'k')
    #im = plt.scatter(lon_inter_in, lat_inter_in, s=10, c=vs_inter_horizon_in[i], cmap='gist_rainbow', vmin=v_min, vmax=v_max)
        im = plt.scatter(lon_inter_in, lat_inter_in, s=10, c=vs_inter_horizon_in[i], cmap='gist_rainbow')
    plt.colorbar(im)
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.tight_layout()
    plt.savefig(dir_image+'Vs_hori_'+str((i+1)*dz)+'m.png',dpi=100)
    plt.close()